In [112]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 30)
types = {'id': int, 'ciudad':str, 'provincia': str}

training_data = pd.read_csv('../../data/TP2/train.csv', dtype = types, usecols=['id','provincia','ciudad','idzona','metrostotales','precio'])
evaluation_data = pd.read_csv('../../data/TP2/test.csv',dtype = types, usecols=['id','provincia','ciudad','idzona','lat','lng'])

In [113]:
training_data.precio.mean()/training_data.metrostotales.mean()

14317.519170109566

In [114]:
metros_ciudad = training_data.groupby('ciudad').agg({'metrostotales':'mean'}).reset_index().rename(columns={'metrostotales':'mts_ciudad'})

meanmts=metros_ciudad.mts_ciudad.mean()
metros_ciudad.mts_ciudad.fillna(meanmts,inplace=True)

training_data = pd.merge(training_data,metros_ciudad, on='ciudad', how='left')
training_data.mts_ciudad.fillna(meanmts,inplace=True)

In [115]:
training_data.metrostotales.fillna(training_data.mts_ciudad,inplace=True)
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240000 entries, 0 to 239999
Data columns (total 7 columns):
id               240000 non-null int32
ciudad           239628 non-null object
provincia        239845 non-null object
metrostotales    240000 non-null float64
idzona           211379 non-null float64
precio           240000 non-null float64
mts_ciudad       240000 non-null float64
dtypes: float64(4), int32(1), object(2)
memory usage: 13.7+ MB


In [116]:
training_data.idzona.fillna(-1, inplace=True)
props_zone = (((training_data.idzona.value_counts()>5)).to_frame().reset_index()).rename(columns={'index':'idzona','idzona':'many_props'})

props_zone.loc[props_zone.idzona==-1] #TENGO QUE CAMBIAR ESTE
#props_zone['+5props'].value_counts()

,idzona,many_props
0,-1.0,True


In [117]:
training_data.ciudad.fillna('no_city', inplace=True)

In [118]:
precio_ciudad = training_data.groupby('ciudad').agg({'precio':'sum','metrostotales':'sum'}).reset_index()
precio_ciudad['precio_mts_ciudad'] = precio_ciudad.precio/precio_ciudad.metrostotales
precio_ciudad.drop(columns={'metrostotales','precio'},inplace=True)
precio_ciudad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876 entries, 0 to 875
Data columns (total 2 columns):
ciudad               876 non-null object
precio_mts_ciudad    876 non-null float64
dtypes: float64(1), object(1)
memory usage: 13.8+ KB


In [119]:
training_data = pd.merge(training_data,props_zone, on='idzona', how='left')
training_data = pd.merge(training_data,precio_ciudad, on= 'ciudad', how='left')
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240000 entries, 0 to 239999
Data columns (total 9 columns):
id                   240000 non-null int32
ciudad               240000 non-null object
provincia            239845 non-null object
metrostotales        240000 non-null float64
idzona               240000 non-null float64
precio               240000 non-null float64
mts_ciudad           240000 non-null float64
many_props           240000 non-null bool
precio_mts_ciudad    240000 non-null float64
dtypes: bool(1), float64(5), int32(1), object(2)
memory usage: 15.8+ MB


In [120]:
training_data.tail()

,id,ciudad,provincia,metrostotales,idzona,precio,mts_ciudad,many_props,precio_mts_ciudad
239995,119879,Zinacantepec,Edo. de México,172.392857,53666.0,650000.0,172.392857,True,8540.403563
239996,259178,Toluca,Edo. de México,250.000000,51954.0,1940000.0,140.260545,True,10332.977275
239997,131932,Benito Juárez,Distrito Federal,138.000000,50003995.0,3400000.0,138.116722,True,24505.790860
239998,146867,Iztapalapa,Distrito Federal,137.000000,24162.0,2890000.0,137.449091,True,12363.602801
239999,121958,Cuajimalpa de Morelos,Distrito Federal,193.322275,23750.0,3650000.0,193.322275,True,24426.596358


In [121]:
precio_zona = training_data.groupby('idzona').agg({'precio':'sum','metrostotales':'sum','many_props':'median'}).reset_index()
precio_zona['precio_por_metro'] = precio_zona.precio/precio_zona.metrostotales
#precio_zona.loc[(~(precio_zona['+5props']))|(precio_zona.idzona==-1), 'precio_por_metro'] = precio_zona.precio_mts_ciudad
precio_zona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19720 entries, 0 to 19719
Data columns (total 5 columns):
idzona              19720 non-null float64
precio              19720 non-null float64
metrostotales       19720 non-null float64
many_props          19720 non-null bool
precio_por_metro    19720 non-null float64
dtypes: bool(1), float64(4)
memory usage: 635.6 KB


In [122]:
precio_zona = precio_zona[['idzona','precio_por_metro','many_props']]

In [123]:
precio_zona.many_props.value_counts()

False    13895
True      5825
Name: many_props, dtype: int64

In [95]:
###precio_zona.to_csv('../../res/ftr/precio_por_metro_idzona.csv')

In [144]:
training_data = pd.read_csv('../../data/TP2/train.csv', dtype = types, usecols=['id','idzona','ciudad'])
evaluation_data = pd.read_csv('../../data/TP2/test.csv',dtype = types, usecols=['id','idzona','ciudad'])

training_data.idzona.fillna(-1, inplace=True)
evaluation_data.idzona.fillna(-1, inplace=True)


In [145]:
evaluation_data=pd.merge(evaluation_data,precio_zona,on='idzona',how='left')
evaluation_data=pd.merge(evaluation_data,precio_ciudad,on='ciudad',how='left')

evaluation_data.precio_mts_ciudad.fillna(evaluation_data.precio_mts_ciudad.mean(),inplace=True)
evaluation_data.precio_por_metro.fillna(evaluation_data.precio_mts_ciudad,inplace=True)


training_data=pd.merge(training_data,precio_zona,on='idzona',how='left')
training_data=pd.merge(training_data,precio_ciudad,on='ciudad',how='left')

training_data.precio_mts_ciudad.fillna(training_data.precio_mts_ciudad.mean(),inplace=True)
training_data.precio_por_metro.fillna(training_data.precio_mts_ciudad,inplace=True)

In [146]:
evaluation_data.head()

,id,ciudad,idzona,precio_por_metro,many_props,precio_mts_ciudad
0,4941,Miguel Hidalgo,-1.0,12129.550196,True,29832.537042
1,51775,Mérida,113851.0,9876.449484,True,7827.774773
2,115253,Coyoacán,23620.0,19383.955540,True,20271.810865
3,299321,Acapulco de Juárez,129347.0,10149.976640,True,15302.447832
4,173570,Tultitlán,57125.0,13780.965274,True,8346.134850


In [147]:
evaluation_data.loc[((evaluation_data['many_props'])==False)|(evaluation_data.idzona==-1), 'precio_por_metro'] = evaluation_data.precio_mts_ciudad
training_data.loc[((training_data['many_props'])==False)|(training_data.idzona==-1), 'precio_por_metro'] = training_data.precio_mts_ciudad


In [148]:
training_data = training_data[['id','precio_por_metro']]
evaluation_data = evaluation_data[['id','precio_por_metro']]

In [151]:
evaluation_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60000 entries, 0 to 59999
Data columns (total 2 columns):
id                  60000 non-null int32
precio_por_metro    60000 non-null float64
dtypes: float64(1), int32(1)
memory usage: 3.6 MB


In [152]:
training_data.to_csv('../../res/ftr/mean_idzona_price_train.csv')
evaluation_data.to_csv('../../res/ftr/mean_idzona_price_evaluation.csv')